In [5]:
import pandas as pd
import os, io
import yaml
import numpy as np

from selenium import webdriver

In [13]:
import httplib2
import io
import argparse
import requests


from httplib2 import Http
from oauth2client import file,client, tools
from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive',
         'https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/spreadsheets.currentonly',
         'https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/presentations',
         'https://www.googleapis.com/auth/drive.scripts',
         'https://www.googleapis.com/auth/drive.appdata',
         'https://www.googleapis.com/auth/drive.metadata'
        ]
store = file.Storage('/Users/daniel/OAN/credentials/storage.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('/Users/daniel/OAN/credentials/storage.json', SCOPES)
    creds = tools.run_flow(flow, store)
drive_service = discovery.build('drive', 'v3', http=creds.authorize(Http()))
sheet_service = discovery.build('sheets', 'v4', http=creds.authorize(Http()))

In [5]:
cred_map='/Users/daniel/OAN/credentials/maps.yaml'

In [6]:
cred_map_yaml=yaml.load( open(cred_map, "r" ),Loader=yaml.FullLoader)

In [9]:
ID=cred_map_yaml['SHEET_ID']

In [8]:
RANGE_NAME = 'Sheet1'
    
gsheet = sheet_service.spreadsheets().values().get(spreadsheetId=ID, range=RANGE_NAME).execute()

df_villages=pd.DataFrame(gsheet.get('values', [])[1:],columns= gsheet.get('values', [])[0:1][0])

## OPEN FIRST VILLAGE

In [9]:
df_villages.loc[1,'Translate']

'SONKOROU'

In [10]:
village=df_villages.loc[1,'Translate']

In [11]:
village

'SONKOROU'

In [14]:
overpass_url = "http://overpass-api.de/api/interpreter"

#When the village is well defined
overpass_query = ("""
[out:json];
area['name'='"""+village+
"""']->.a;
way[building](area.a);
(._;>;);
out body;
""")

#When the village is a dot or something
overpass_query_2 = ("""
[out:json];
way['name'='"""+village+
"""'];
way(around:1000)[building];
(._;>;);
out body;
""")

#get Data
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
#response_2 = requests.get(overpass_url, 
#                        params={'data': overpass_query_2})
data = response.json()
#data = response_2.json()

In [15]:
houses = []
ways=[elem for elem in data['elements']  if elem['type']=='way']
nodes=[elem for elem in data['elements']  if elem['type']=='node']
for way in ways:
    houses_info={}
    houses_info['id']=way['id']
    nodes_lats=[]
    nodes_longs=[]
    houses_info['nodes']=way['nodes']
    for node in nodes:
        if node['id'] in houses_info['nodes']:
            nodes_lats.append(node['lat'])
            nodes_longs.append(node['lon'])
    houses_info['lat']=np.mean(np.array(nodes_lats))
    houses_info['lon']=np.mean(np.array(nodes_longs))
    houses.append(houses_info)

In [16]:
df_houses=pd.DataFrame(houses)

In [17]:
len(df_houses)

22

In [18]:
df_houses.head(4)

,id,nodes,lat,lon
0,741925935,"[6945452471, 6945452472, 6945452473, 694545247...",9.835842,3.340055
1,741925936,"[6945452475, 6945452476, 6945452477, 694545247...",9.835694,3.340124
2,741925937,"[6945452479, 6945452480, 6945452481, 694545248...",9.835600,3.339915
3,741925938,"[6945452483, 6945452484, 6945468885, 694546888...",9.835705,3.340827


In [19]:
#Mapbox
#center=(df_houses.lon.mean(),df_houses.lat.mean())
#googleMaps
center=(df_houses.lat.mean(),df_houses.lon.mean())

## open data in ESRI

In [1]:
from arcgis.gis import GIS

In [29]:
cred_map='/Users/daniel/OAN/credentials/maps.yaml'

In [30]:
cred_map_yaml=yaml.load( open(cred_map, "r" ),Loader=yaml.FullLoader)

In [31]:
os.environ["ESRI_PASS"]=cred_map_yaml["ESRI_PASS"]

In [3]:
gis = GIS(profile='dani_oan', password=os.environ["ESRI_PASS"])

In [41]:
ex_map = gis.map(zoomlevel=17)

In [42]:
ex_map.layout={'height':'600px','width':'800px'}

In [43]:
ex_map.center =center

In [44]:
ex_map.basemap = 'satellite'

In [45]:
ex_map

MapView(layout=Layout(height='600px', width='800px'), zoom=17.0)

# EXPORT MAP

# HTML

In [47]:
ex_map.export_to_html('gmap.html')

True

# Telecharget l'image

In [50]:
driver = webdriver.Chrome(executable_path=
                          r'/Users/daniel/OAN/nikki_map_house_count/maps_scrapping/chromedriver')

In [51]:
file = "file:/Users/daniel/OAN/nikki_map_house_count/maps_scrapping/gmap.html"

In [52]:
driver.get(file)

In [55]:
screen=driver.save_screenshot(village+'.png')

## CUT IMAGE

In [56]:
from PIL import Image

In [57]:
def crop(image_path, coords, saved_location):
    """
    @param image_path: The path to the image to edit
    @param coords: A tuple of x/y coordinates (x1, y1, x2, y2)
    @param saved_location: Path to save the cropped image
    """
    image_obj = Image.open(image_path)
    cropped_image = image_obj.crop(coords)
    cropped_image.save(saved_location)

In [58]:
cut="/Users/daniel/OAN/nikki_map_house_count/maps_scrapping/"+village+'2.png'

In [70]:
crop(village+'.png',(100, 100, 1600, 1300),village+'2.png')